In [43]:
import os
import pandas as pd
import datetime as dt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from api_credentials import client_id, client_secret, redirect_uri
import json

# Load playlists from JSON file
with open('playlists.json', 'r') as file:
    playlists_dict = json.load(file)


# Initialize the Spotify client with client credentials for public data access
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Read in the AU Daily 200 Spotify Chart  
Manually downloaded CSV from https://charts.spotify.com/charts/view/regional-au-daily/latest

In [44]:
# Load the CSV file
file_path = 'raw_data/regional-au-daily-2024-02-17.csv'
df = pd.read_csv(file_path)

# Extract the date from the file name
file_name = os.path.basename(file_path)  
date_str = file_name.split('-')[-3:]  # Splits the file name and takes the last 3 elements ['2024', '02', '05.csv']
date_str = '-'.join(date_str)  # Joins them back into '2024-02-05.csv'
date = date_str.split('.')[0]  # Removes the file extension, resulting in '2024-02-05'

# Convert the date string to a datetime object
date_datetime = pd.to_datetime(date)

# Format the datetime object to a string in 'DD-MM-YYYY' format
formatted_date = date_datetime.strftime('%d-%m-%Y')

# Add the formatted date as a new column to the DataFrame
df['date'] = formatted_date

Extract each tracks unique id for API call 

In [45]:
track_ids = df['uri'].str[14:]
df['track_id'] = df['uri'].str[14:]
df.drop(columns='uri', inplace=True)

Call Spotify API to retrieve `popularity_score` for each of the top 200 tracks

In [46]:
# Convert the Series to a list for processing
track_ids_list = track_ids.tolist()

# Spotify's `tracks` endpoint can take multiple track IDs at once,
# but to avoid potential rate limits or payload size issues,
# it's a good idea to process them in batches.
# Here, we define a batch size (e.g., 50 tracks per request)
batch_size = 50

# Initialise lists to store the retrieved data
artists = []
track_names = []
popularity_scores = []

for i in range(0, len(track_ids_list), batch_size):
    batch = track_ids_list[i:i+batch_size]
    tracks_info = sp.tracks(batch)
    for track in tracks_info['tracks']:
        # Some tracks might not be available, leading to `None` entries
        if track:
            # Extracting the primary artist name
            artist_name = track['artists'][0]['name'] if track['artists'] else 'N/A'
            artists.append(artist_name)
            
            # Extracting the track name
            track_names.append(track['name'])
            
            # Extracting the popularity score
            popularity_scores.append(track['popularity'])

# Now that we have all the data, put it into a DataFrame
popularity_df = pd.DataFrame({
    'artist_name': artists,
    'track_name': track_names,
    'popularity_score': popularity_scores
})

# Show the DataFrame 
popularity_df.head()


,artist_name,track_name,popularity_score
0,Taylor Swift,Cruel Summer,99
1,Noah Kahan,Stick Season,98
2,Benson Boone,Beautiful Things,98
3,Jack Harlow,Lovin On Me,99
4,Beyoncé,TEXAS HOLD 'EM,55


In [47]:
df['popularity_score'] = popularity_df['popularity_score']

In [48]:
# Reorder columns
df = df[['date', 'rank', 'artist_names', 'track_name', 'track_id', 'source', 'peak_rank', 'previous_rank', 'days_on_chart', 'streams', 'popularity_score']]

# Rename columns
df.rename(columns={
    'source': 'label',
    'streams': 'daily_streams',
}, inplace=True)

In [49]:
df.head()

,date,rank,artist_names,track_name,track_id,label,peak_rank,previous_rank,days_on_chart,daily_streams,popularity_score
0,17-02-2024,1,Taylor Swift,Cruel Summer,1BxfuPKGuaTgP7aM0Bbdwr,Taylor Swift,1,1,378,434183,99
1,17-02-2024,2,Noah Kahan,Stick Season,0mflMxspEfB0VbI1kyLiAv,Mercury Records/Republic Records,1,2,135,291252,98
2,17-02-2024,3,Benson Boone,Beautiful Things,6tNQ70jh4OwmPGpYy6R2o9,"Night Street Records, Inc./Warner Records Inc.",2,3,30,280357,98
3,17-02-2024,4,Jack Harlow,Lovin On Me,4xhsWYTOGcal8zt0J161CU,Generation Now/Atlantic,1,5,100,267887,99
4,17-02-2024,5,Beyoncé,TEXAS HOLD 'EM,7AYmToZ5y67fBjq4RLFbuq,Parkwood Entertainment/Columbia,4,4,6,254801,55


Retrieve additional data from Kworb.net
This will give us additional features/columns like:  
 - `count_at_peak`
 - `rank_movement`
 - `streams+`
 - `7day_streams`
 - `7day_streams_movement` 
 - `total_streams`

In [50]:
# Read the HTML table into a DataFrame
kworb_df = pd.read_html('https://kworb.net/spotify/country/au_daily.html')[0]

# Get the current date minus 2 days to account for data lag
current_date_minus_2 = dt.datetime.now().date() - dt.timedelta(days=2)

# Add the current date as a new column to the DataFrame
kworb_df['DATE'] = current_date_minus_2

In [51]:
kworb_df = kworb_df.rename(columns={
    'Pos' : 'rank',
    'P+': 'rank_change',
    'Artist and Title' : 'artist_title',
    'Days' : 'days_on_chart',
    'Pk' : 'peak_rank',
    '(x?)' : 'days_at_peak',
    'Streams' : 'daily_streams',
    'Streams+' : 'daily_streams_change',
    '7Day' : '7Day_streams',
    '7Day+' : '7Day_streams_change',
    'Total' : 'total_streams',
})

Need to change "=" to "0", remove "+", and handle "RE"

In [52]:
# Remove "+" and ensure "=" is replaced with "0"
kworb_df['rank_change'] = kworb_df['rank_change'].str.replace('+', '').str.replace('=', '0', regex=False)

# Attempt to convert the column to numeric without coercing errors to identify potential non-numeric values
try:
    kworb_df['rank_change'] = pd.to_numeric(kworb_df['rank_change'])
    print("Conversion successful. No NaN values detected.")
except ValueError as e:
    print("Conversion failed due to non-numeric values:", e)

# Optionally, inspect the column for any remaining non-numeric values or anomalies
# This step is just for verification and won't change the DataFrame
non_numeric = kworb_df[pd.to_numeric(kworb_df['rank_change'], errors='coerce').isna()]
if not non_numeric.empty:
    print("Non-numeric values found:\n", non_numeric)
else:
    print("No non-numeric values found.")

Conversion failed due to non-numeric values: Unable to parse string "NEW" at position 156
Non-numeric values found:
      rank rank_change                                       artist_title  \
156   157         NEW                             Djo - End of Beginning   
178   179          RE                     Van Morrison - Brown Eyed Girl   
180   181          RE  Tears For Fears - Everybody Wants To Rule The ...   
184   185          RE  Taylor Swift - Now That We Don't Talk (Taylor'...   
185   186          RE  Timbaland - The Way I Are (w/ Keri Hilson, D.O...   
189   190          RE                   Bruno Mars - When I Was Your Man   
190   191          RE     Journey - Don't Stop Believin' (2022 Remaster)   
194   195          RE             Empire of the Sun - Walking On A Dream   
197   198         NEW       P!nk - Just Give Me a Reason (w/ Nate Ruess)   

     days_on_chart  peak_rank days_at_peak  daily_streams  \
156              1        157          NaN          56477   


/var/folders/px/x65fp4pn31s5cv4gsbh_3cl80000gn/T/ipykernel_3030/641918597.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  kworb_df['rank_change'] = kworb_df['rank_change'].str.replace('+', '').str.replace('=', '0', regex=False)


In [53]:
# Extract numeric values from 'COUNT_AT_PEAK'
kworb_df['days_at_peak'] = kworb_df['days_at_peak'].str.extract('(\d+)', expand=False)

# Convert to numeric, allowing NaNs to remain
kworb_df['days_at_peak'] = pd.to_numeric(kworb_df['days_at_peak'], errors='coerce')

In [54]:
columns_to_keep = kworb_df[['artist_title', 'rank_change', 'days_at_peak', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams']]
columns_to_keep.head(3)

,artist_title,rank_change,days_at_peak,daily_streams_change,7Day_streams,7Day_streams_change,total_streams
0,Taylor Swift - Cruel Summer,0,4.0,8597.0,2231385,169209,54869471
1,Noah Kahan - Stick Season,0,19.0,-20733.0,2189989,-24018,24873764
2,Benson Boone - Beautiful Things,0,7.0,-30155.0,2047978,17828,6461353


In [55]:
print(f"Spotify Daily 200 columns:", df.columns.tolist())
print(f"Kworb Daily 200 columns:", kworb_df.columns.tolist())

Spotify Daily 200 columns: ['date', 'rank', 'artist_names', 'track_name', 'track_id', 'label', 'peak_rank', 'previous_rank', 'days_on_chart', 'daily_streams', 'popularity_score']
Kworb Daily 200 columns: ['rank', 'rank_change', 'artist_title', 'days_on_chart', 'peak_rank', 'days_at_peak', 'daily_streams', 'daily_streams_change', '7Day_streams', '7Day_streams_change', 'total_streams', 'DATE']


In [56]:
df['rank_change'] = kworb_df['rank_change']
df['days_at_peak'] = kworb_df['days_at_peak']
df['daily_streams_change'] = kworb_df['daily_streams_change']
df['7Day_streams'] = kworb_df['7Day_streams']
df['7Day_streams_change'] = kworb_df['7Day_streams_change']
df['total_streams'] = kworb_df['total_streams']


In [57]:
columns_order =  [
    'date',
    'artist_names',
    'track_name',
    'rank',
    'previous_rank',
    'rank_change',
    'peak_rank',
    'days_at_peak',
    'days_on_chart',
    'popularity_score',
    'daily_streams',
    'daily_streams_change',
    '7Day_streams',
    '7Day_streams_change',
    'total_streams',
    'label',
    'track_id',
]

df = df[columns_order]

In [58]:
df.head()

,date,artist_names,track_name,rank,previous_rank,rank_change,peak_rank,days_at_peak,days_on_chart,popularity_score,daily_streams,daily_streams_change,7Day_streams,7Day_streams_change,total_streams,label,track_id
0,17-02-2024,Taylor Swift,Cruel Summer,1,1,0,1,4.0,378,99,434183,8597.0,2231385,169209,54869471,Taylor Swift,1BxfuPKGuaTgP7aM0Bbdwr
1,17-02-2024,Noah Kahan,Stick Season,2,2,0,1,19.0,135,98,291252,-20733.0,2189989,-24018,24873764,Mercury Records/Republic Records,0mflMxspEfB0VbI1kyLiAv
2,17-02-2024,Benson Boone,Beautiful Things,3,3,0,2,7.0,30,98,280357,-30155.0,2047978,17828,6461353,"Night Street Records, Inc./Warner Records Inc.",6tNQ70jh4OwmPGpYy6R2o9
3,17-02-2024,Jack Harlow,Lovin On Me,4,5,1,1,72.0,100,99,267887,-10696.0,1841663,-8583,31467425,Generation Now/Atlantic,4xhsWYTOGcal8zt0J161CU
4,17-02-2024,Beyoncé,TEXAS HOLD 'EM,5,4,-1,4,NaN,6,55,254801,-26643.0,1280930,254801,1280930,Parkwood Entertainment/Columbia,7AYmToZ5y67fBjq4RLFbuq


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  200 non-null    object 
 1   artist_names          200 non-null    object 
 2   track_name            200 non-null    object 
 3   rank                  200 non-null    int64  
 4   previous_rank         200 non-null    int64  
 5   rank_change           200 non-null    object 
 6   peak_rank             200 non-null    int64  
 7   days_at_peak          45 non-null     float64
 8   days_on_chart         200 non-null    int64  
 9   popularity_score      200 non-null    int64  
 10  daily_streams         200 non-null    int64  
 11  daily_streams_change  191 non-null    float64
 12  7Day_streams          200 non-null    int64  
 13  7Day_streams_change   200 non-null    int64  
 14  total_streams         200 non-null    int64  
 15  label                 2

In [60]:
yesterday_df = pd.read_csv('combined_data/combined.csv')
yesterday_df.tail()

,date,artist_names,track_name,rank,previous_rank,rank_change,peak_rank,days_at_peak,days_on_chart,popularity_score,daily_streams,daily_streams_change,7Day_streams,7Day_streams_change,total_streams,label,track_id
2395,16/2/2024,Tom Odell,Another Love,196,176,-20,18,NaN,790,67,51392,-1728.0,306386,1771,47885840,ITNO/Columbia,7jtQIBanIiJOMS6RyCx6jZ
2396,16/2/2024,SZA,Nobody Gets Me,197,158,-39,12,NaN,116,86,50991,-5922.0,426312,-24277,8247659,Top Dawg Entertainment/RCA Records,5Y35SjAfXjjG0sFQ3KOxmm
2397,16/2/2024,"Metro Boomin, The Weeknd, 21 Savage",Creepin' (with The Weeknd & 21 Savage),198,200,2,2,4.0,440,91,50922,183.0,355090,-2866,45933576,Republic Records,2dHHgzDwk4BJdRwy9uXhTO
2398,16/2/2024,"¥$, Kanye West, Ty Dolla $ign",VULTURES,199,146,-53,29,NaN,6,26,50902,-8238.0,110042,50902,110042,Ye,3SIRBp4RRQ2AO5H4NO7xfq
2399,16/2/2024,Arctic Monkeys,I Wanna Be Yours,200,191,-9,63,NaN,532,96,50892,-988.0,357508,50892,25296198,Domino Recording Co,5XeFesFbtLpXzIVDNQP22n


In [61]:
combined_df = pd.concat([yesterday_df, df], ignore_index=True)

In [62]:
print(len(combined_df))

2600


In [63]:
combined_df.to_csv('combined_data/combined.csv', index=False)